In [1]:
!pip install flask pyngrok transformers torch

import os
import torch
import torch.nn.functional as F
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import google.generativeai as genai
from google.colab import userdata

try:
    __import__('google.generativeai')
except ImportError:
    !pip install -q google-generativeai

emo_tokenizer = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
emo_model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [2]:
try:
    NGROK_AUTHTOKEN = userdata.get('ngrok_key')
    GEMINI_API_KEY = userdata.get('gemini_api_key')
    if not NGROK_AUTHTOKEN:
        raise ValueError("ngrok_key not found in userdata. Please add it.")
    if not GEMINI_API_KEY:
        raise ValueError("gemini_api_key not found in userdata. Please add it.")

    !ngrok config add-authtoken $NGROK_AUTHTOKEN

    genai.configure(api_key=GEMINI_API_KEY)
except ValueError as e:
    print(f"Error: {e}")
    exit()

app = Flask(__name__)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
@app.route("/predict", methods=["POST"])
def predict():
    text = request.json.get("text", "")

    if not text:
        return jsonify({"error": "No text provided"}), 400

    emo_inputs = emo_tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        emo_outputs = emo_model(**emo_inputs)
        logits = emo_outputs.logits
        probs = F.softmax(logits, dim=-1)

    top_k = 5
    top_probs, top_indices = torch.topk(probs, k=top_k, dim=-1)

    emotions = []
    for score, idx in zip(top_probs[0], top_indices[0]):
        label = emo_model.config.id2label[idx.item()]
        emotions.append({"label": label, "score": float(score.item())})

    top_emotion = emotions[0]["label"]

    prompt = f"""
    You are a compassionate and empathetic conversational AI.

    User said: "{text}"
    Detected emotion: {top_emotion}

    Task:
    - Reply in first person (using "I" and "me").
    - Your reply MUST be based on validating the user's emotion.
    - Your tone must be warm, supportive, and gentle.
    - If user is happy converse with them hear their story and make conversations with them.
    - If the user's emotion is 'sadness' or 'grief', you can gently suggest talking to people like friends, family, or professionals.
    - Keep the reply short and natural, like a friend would.
    - DO NOT give any other advice, solutions, or suggestions.
    - Focus on acknowledging their feeling and offering to listen.
    - If the emotion is sadness, do NOT use language that could be interpreted as self-harm or despair.
    """

    try:
        gemini_model = genai.GenerativeModel('gemini-2.5-flash-preview-05-20')
        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.GenerationConfig(
                temperature=0.7,
                top_p=0.9,
                top_k=40,
                max_output_tokens=100
            )
        )
        reply = response.text
    except Exception as e:
        print(f"API Error: {e}")
        if top_emotion in ['sadness', 'grief']:
            reply = "I hear you. Sometimes it helps to talk to friends or family about what you're going through. I'm here to listen, too."
        else:
            reply = "I hear you. I'm here if you want to talk more about it."

    return jsonify({
        "input": text,
        "emotions": emotions,
        "reply": reply
    })

In [4]:
try:
    public_url = ngrok.connect(5000)
    print("Public URL:", public_url)

    app.run(port=5000, use_reloader=False)
except Exception as e:
    print(f"Failed to create ngrok tunnel or run the app: {e}")

Public URL: NgrokTunnel: "https://5ad5f9fface3.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Sep/2025 19:20:34] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Sep/2025 19:21:20] "POST /predict HTTP/1.1" 200 -


API Error: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 2.


INFO:werkzeug:127.0.0.1 - - [20/Sep/2025 19:22:11] "POST /predict HTTP/1.1" 200 -


API Error: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 2.


# > This is the output that we get once we do an API test.


    "input": "I am really sad right now",
    "emotions": [
        {
            "label": "sadness",
            "score": 0.9742826819419861
        },
        {
            "label": "disappointment",
            "score": 0.006741698365658522
        },
        {
            "label": "grief",
            "score": 0.0031700627878308296
        },
        {
            "label": "neutral",
            "score": 0.002348646055907011
        },
        {
            "label": "remorse",
            "score": 0.0016634233761578798
        }
    ],
    "reply": "I hear you. Sometimes it helps to talk to friends or family about what you're going through. I'm here to listen, too."